# 端到端建模流程
## 从原始数据输入到结果文件生成

# 0.引入三方库

In [31]:
import gc
import numpy as np
import pandas as pd

import lightgbm as lgb

# 1.读取所需数据

In [2]:
debug = False
NROWS = 10000 if debug else None
train = pd.read_csv('data/train.csv', nrows = NROWS)
test =  pd.read_csv('data/test.csv', nrows = NROWS)

merchant = pd.read_csv('data/merchants.csv', nrows = NROWS)
new_transaction = pd.read_csv('data/new_merchant_transactions.csv', nrows = NROWS)
history_transaction = pd.read_csv('data/historical_transactions.csv', nrows = NROWS)

# 2.做数据预处理

In [3]:
# 商户信息
def change_object_cols(se):
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value)), index=value)).values

In [4]:
category_cols = ['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'category_4', 'city_id', 'state_id', 'category_2']
numeric_cols = ['numerical_1', 'numerical_2',
     'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12']

# 将离散字段进行编码
for col in ['category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']:
    merchant[col] = change_object_cols(merchant[col])
    
# 同时为了能够更方便统计，首先做缺失值处理，对离散字段可统一用-1进行填充
merchant[category_cols] = merchant[category_cols].fillna(-1)

# 对离散型字段探查发现有正无穷值，这也是模型所不能接受的，因此需要对无限值进行处理
inf_cols = ['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12']
merchant[inf_cols] = merchant[inf_cols].replace(np.inf, merchant[inf_cols].replace(np.inf, -99).max().max())

# 对于离散字段的缺失值处理方式也有多样，这里先使用平均值进行填充，后续有需要再进行优化处理
for col in numeric_cols:
    merchant[col] = merchant[col].fillna(merchant[col].mean())
    
# 去除与transaction重复的列(当然, id显然不包括在内)
duplicate_cols = ['merchant_id', 'merchant_category_id', 'subsector_id', 'category_1', 'city_id', 'state_id', 'category_2']

In [5]:
merchant = merchant.drop(duplicate_cols[1:], axis=1)

merchant = merchant.loc[merchant['merchant_id'].drop_duplicates().index.tolist()].reset_index(drop=True)

In [6]:
# 交易记录
transaction = pd.concat([new_transaction, history_transaction], axis=0, ignore_index=True)
del new_transaction
del history_transaction
gc.collect()

numeric_cols = [ 'installments', 'month_lag', 'purchase_amount']
category_cols = ['authorized_flag', 'card_id', 'city_id', 'category_1',
       'category_3', 'merchant_category_id', 'merchant_id', 'category_2', 'state_id',
       'subsector_id']
time_cols = ['purchase_date']

In [7]:
# 可仿照merchant的处理方式对字符型的离散特征进行编码（id除外）
for col in ['authorized_flag', 'category_1', 'category_3']:
    transaction[col] = change_object_cols(transaction[col].fillna(-1).astype(str))
transaction[category_cols] = transaction[category_cols].fillna(-1)
transaction['category_2'] = transaction['category_2'].astype(int)
# 进行时间段的处理，简单起见进行月份、日期的星期数（工作日与周末）、以及
# 时间段（上午、下午、晚上、凌晨）进行提取

In [8]:
from datetime import datetime
transaction['purchase_month'] = transaction['purchase_date'].apply(lambda x:'-'.join(x.split(' ')[0].split('-')[:2]))
transaction['purchase_hour_section'] = transaction['purchase_date'].apply(lambda x: x.split(' ')[1].split(':')[0]).astype(int)//6
transaction['purchase_day'] = transaction['purchase_date'].apply(lambda x: datetime.strptime(x.split(" ")[0], "%Y-%m-%d").weekday())//5                                                                    

In [9]:
del transaction['purchase_date']

In [10]:
# 减少内存起见，统一对merchant和transaction的id列进行编码
merchant_id = set(merchant['merchant_id'].unique().tolist() + transaction['merchant_id'].unique().tolist())
merchant_map = pd.Series(range(len(merchant_id)), index=merchant_id)

transaction['merchant_id'] = transaction['merchant_id'].map(merchant_map)
merchant['merchant_id'] = merchant['merchant_id'].map(merchant_map)

In [12]:
# 减少内存起见，统一对merchant和transaction的id列进行编码

se_map = change_object_cols(train['first_active_month'].append(test['first_active_month']).astype(str))

train['first_active_month'] = se_map[:train.shape[0]]
test['first_active_month'] = se_map[train.shape[0]:]



In [13]:
## 对购买月份进行编码
transaction['purchase_month'] = change_object_cols(transaction['purchase_month'])

In [14]:
transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'category_2', 'state_id', 'subsector_id',
       'purchase_month', 'purchase_hour_section', 'purchase_day'],
      dtype='object')

In [15]:
## 初步策略，第一步采用字典结构统计用户的所有行为特征
## 第二步，用户层面横向进行比较和特征提取
## 第三步，纵向对比所有用户的交易记录
cols = ['merchant_id', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
transaction = pd.merge(transaction, merchant[cols], how='left', on='merchant_id')
numeric_cols = ['purchase_amount', 'installments']
category_cols = ['authorized_flag', 'city_id', 'category_1',
       'category_3', 'merchant_category_id','month_lag','most_recent_sales_range',
                 'most_recent_purchases_range', 'category_4',
                 'purchase_month', 'purchase_hour_section', 'purchase_day']
id_cols = ['card_id', 'merchant_id']

In [16]:
transaction[cols[1:]] = transaction[cols[1:]]

In [17]:
transaction[category_cols].nunique()

authorized_flag                  2
city_id                        308
category_1                       2
category_3                       4
merchant_category_id           331
month_lag                       16
most_recent_sales_range          5
most_recent_purchases_range      5
category_4                       2
purchase_month                  16
purchase_hour_section            4
purchase_day                     2
dtype: int64

In [18]:
transaction[category_cols] =transaction[category_cols]

# 3.进行特征提取

In [19]:
transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'category_2', 'state_id', 'subsector_id',
       'purchase_month', 'purchase_hour_section', 'purchase_day',
       'most_recent_sales_range', 'most_recent_purchases_range', 'category_4'],
      dtype='object')

In [20]:
numeric_cols = ['authorized_flag',  'category_1', 'installments',
       'category_3',  'month_lag',
       'purchase_amount', 'category_2', 
       'purchase_month', 'purchase_hour_section', 'purchase_day',
       'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
categorical_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id', ]
aggs = {}
for col in numeric_cols:
    aggs[col] = ['nunique', 'mean', 'min', 'max','var','skew', 'sum']
for col in categorical_cols:
    aggs[col] = ['nunique']    
aggs['card_id'] = ['size', 'count']

cols = ['card_id']
for key in aggs.keys():
    cols.extend([key+'_'+stat for stat in aggs[key]])

In [21]:
df = transaction[transaction['month_lag']<0].groupby('card_id').agg(aggs).reset_index()
df.columns = cols[:1] + [co+'_hist' for co in cols[1:]]
df2 = transaction[transaction['month_lag']>=0].groupby('card_id').agg(aggs).reset_index()
df2.columns = cols[:1] + [co+'_new' for co in cols[1:]]
df = pd.merge(df, df2, how='left',on='card_id')
df2 = transaction.groupby('card_id').agg(aggs).reset_index()
df2.columns = cols
df = pd.merge(df, df2, how='left',on='card_id')

In [22]:
del transaction
gc.collect()

253

# 4.进行模型训练及预测

In [ ]:
train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df
train.to_csv("preprocess/train_groupby.csv", index=False)
test.to_csv("preprocess/test_groupby.csv", index=False)


In [32]:
train = pd.read_csv("preprocess/train_groupby.csv")
test = pd.read_csv("preprocess/test_groupby.csv")
label = 'target'
features = train.columns.tolist()
features.remove('card_id')
features.remove('target')

train = train.sample(frac=1, random_state=2020)
valid = train[int(0.8*train.shape[0]):].reset_index(drop=True)
train = train[:int(0.8*train.shape[0])].reset_index(drop=True)

# 配置模型的训练参数
params_initial = {
    'num_leaves':31, 
    'learning_rate':0.1, 
'boosting':'gbdt',
'min_child_samples':20,
'bagging_seed': 2020,
'bagging_fraction':0.7, 
'bagging_freq':1,
'feature_fraction':0.7, 
 'max_depth':-1,
    'metric':'rmse',
'reg_alpha':0,
'reg_lambda':1, 
'objective':'regression'
    }
ESR = 30
NBR = 10000
VBE = 50

# 转换训练集与验证集为指定的数据格式
train = lgb.Dataset(train[features], train[label])
valid = lgb.Dataset(valid[features], valid[label])

# 模型训练
bst = lgb.train(params_initial,train, 
      num_boost_round=NBR, valid_sets=[train,valid],
      valid_names=['train','valid'],early_stopping_rounds=ESR,verbose_eval=VBE)

# 预测结果
prediction = bst.predict(test[features])
# 获取模型的特征重要性
fse = pd.Series(bst.feature_importance(), index=features)


test['target'] = prediction

submission = test[['card_id', 'target']]

In [25]:
submission.to_csv('result/baseline_groupby_submission.csv', index=False)

In [26]:
fse.to_csv('result/baseline_groupby_feature_importance.csv', index=False)

In [30]:
pd.read_csv("preprocess/train_groupby.csv").head()

first_active_month          card_id  feature_1  feature_2  feature_3  \
0                  67  C_ID_92a2005557          5          2          1   
1                  62  C_ID_3d0044924f          4          1          0   
2                  57  C_ID_d639edf6cd          2          2          0   
3                  70  C_ID_186d6a6901          4          3          0   
4                  72  C_ID_cdbd2c0db2          1          3          0   

     target  authorized_flag_nunique_hist  authorized_flag_mean_hist  \
0 -0.820283                             2                   0.945148   
1  0.392913                             2                   0.963211   
2  0.688056                             2                   0.952381   
3  0.142495                             1                   1.000000   
4 -0.159749                             2                   0.979592   

   authorized_flag_min_hist  authorized_flag_max_hist      ...        \
0                         0                         1      ...         
1                         0                         1      ...         
2                         0                         1      ...         
3                         1                         1      ...         
4                         0                         1      ...         

   category_4_var  category_4_skew  category_4_sum  city_id_nunique  \
0        0.054623        -3.811953           261.0                9   
1        0.075036        -3.073118           327.0                9   
2        0.065011        -3.548480            41.0                5   
3        0.023523        -6.361110            82.0                7   
4        0.091496        -2.668681           151.0                7   

   merchant_category_id_nunique  merchant_id_nunique  state_id_nunique  \
0                            46                  118                 3   
1                            58                  148                 3   
2                             9                   14                 2   
3                            28                   57                 5   
4                            37                  103                 7   

   subsector_id_nunique  card_id_size  card_id_count  
0                    21           283            283  
1                    24           356            356  
2                     8            44             44  
3                    15            84             84  
4                    19           169            169  

[5 rows x 300 columns]

In [27]:
# 后续优化方向
# 1. 只用最近的交易记录(效果较差)
# 2. 特征选择(LGB用特征重要性cv，RF用相关系数等进行筛选，XGB用Stacking进行降维)
# 3. 五折
# 4. 调参（LGB用超参调优，RF用GridSearch）
# 5. 模型融合(LGB+RF+XGB,类别特征少，因此不用CATBOOST)

In [28]:
fse.shape

(298,)

In [40]:
import pandas as pd
data =pd.read_csv("result/LGB_ParamSearch_groupby_submission.csv")

data2 =pd.read_csv("result/XGB_ParamSearch_submission.csv")

data['target2'] = data2['target'].values
print(data.corr())

data['target'] = data['target']*0.6+data['target2']*0.4
data[['card_id','target']].to_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6.csv", index=False)

           target   target2
target   1.000000  0.853771
target2  0.853771  1.000000


In [59]:
train = pd.read_csv("preprocess/train_groupby.csv")
test = pd.read_csv("preprocess/test_groupby.csv")
label = 'target'

In [48]:
train[label] = (train[label]<=-33).astype(int)
train[label].value_counts()

0    199710
1      2207
Name: target, dtype: int64

In [61]:
train[label].min()

-33.21928095

In [49]:
features = train.columns.tolist()
features.remove('card_id')
features.remove('target')

train = train.sample(frac=1, random_state=2020)
valid = train[int(0.8*train.shape[0]):].reset_index(drop=True)
train = train[:int(0.8*train.shape[0])].reset_index(drop=True)

# 配置模型的训练参数
params_initial = {
    'num_leaves':31, 
    'learning_rate':0.1, 
'boosting':'gbdt',
'min_child_samples':20,
'bagging_seed': 2020,
'bagging_fraction':0.7, 
'bagging_freq':1,
'feature_fraction':0.7, 
 'max_depth':-1,
    'metric':'auc',
'reg_alpha':0,
'reg_lambda':1, 
'objective':'binary'
    }
ESR = 30
NBR = 10000
VBE = 50

# 转换训练集与验证集为指定的数据格式
train = lgb.Dataset(train[features], train[label])
valid = lgb.Dataset(valid[features], valid[label])

# 模型训练
bst = lgb.train(params_initial,train, 
      num_boost_round=NBR, valid_sets=[train,valid],
      valid_names=['train','valid'],early_stopping_rounds=ESR,verbose_eval=VBE)

# 预测结果
prediction = bst.predict(test[features])
# 获取模型的特征重要性
fse = pd.Series(bst.feature_importance(), index=features)


test['target'] = prediction

submission = test[['card_id', 'target']]

Training until validation scores don't improve for 30 rounds
[50]	train's auc: 0.96184	valid's auc: 0.894763
Early stopping, best iteration is:
[58]	train's auc: 0.967434	valid's auc: 0.895269


In [52]:
pd.Series(prediction).describe()

count    123623.000000
mean          0.010682
std           0.033903
min           0.000194
25%           0.001012
50%           0.002141
75%           0.007285
max           0.725527
dtype: float64

In [66]:
import pandas as pd
sub1 = pd.read_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6.csv")
sub2 = pd.read_csv("result/LGB_ParamSearch_groupby_Class_submission.csv")
sub = sub1.copy()
for k in range(30, 100, 10):
    card = sub2.sort_values("target", ascending=False)[:k]['card_id'].values.tolist()
    sub['target2'] = sub['card_id'].map(pd.Series(-33.21928095, index=card)).fillna(0)
    sub['target'] = sub['target']*((sub['target2']==0).astype(int)) + sub['target2']
    print(sub['target'].value_counts().head(1))
    sub[sub1.columns].to_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6_top"+str(k)+".csv", index=False)

-33.219281    30
Name: target, dtype: int64
-33.219281    40
Name: target, dtype: int64
-33.219281    50
Name: target, dtype: int64
-33.219281    60
Name: target, dtype: int64
-33.219281    70
Name: target, dtype: int64
-33.219281    80
Name: target, dtype: int64
-33.219281    90
Name: target, dtype: int64


In [67]:
sub2.sort_values("target", ascending=False)

card_id    target
78078   C_ID_922f9c5ea6  0.440025
5008    C_ID_be92f84f5c  0.435115
5009    C_ID_695b445408  0.430665
63978   C_ID_4cf141070f  0.422558
32446   C_ID_ac114ef831  0.419832
120151  C_ID_3e3bb028d1  0.419825
5656    C_ID_126a403be9  0.416189
42391   C_ID_a06a720133  0.393307
109705  C_ID_d596794318  0.391111
27436   C_ID_493de71141  0.390656
77297   C_ID_b0c41ca140  0.390511
43763   C_ID_74dd007aec  0.389445
119449  C_ID_3d40705001  0.388711
64771   C_ID_bc8062c7a5  0.386952
88371   C_ID_26f775a95b  0.386385
65911   C_ID_7d3b40444e  0.384092
25739   C_ID_99db441486  0.384011
104991  C_ID_86ddafb51c  0.381700
114462  C_ID_2a64f34854  0.380729
74063   C_ID_b248952695  0.378909
104735  C_ID_6e089361a2  0.377848
16074   C_ID_9c760806b5  0.376723
69317   C_ID_52fe206a14  0.375704
44743   C_ID_bc1b6ba5c2  0.373448
32027   C_ID_089adf6720  0.372742
83378   C_ID_8a93017bd2  0.370113
99474   C_ID_6e22048f93  0.369138
40613   C_ID_f337ea6293  0.368807
61930   C_ID_fa62ee3693  0.367926
44491   C_ID_75519a6d74  0.365031
...                 ...       ...
24072   C_ID_2bde1975ad  0.000627
57063   C_ID_e2ef8338d8  0.000627
101625  C_ID_bddc4810d2  0.000627
123018  C_ID_94c7b8af20  0.000627
123117  C_ID_0523de956e  0.000627
61255   C_ID_fde82b1812  0.000627
38065   C_ID_8308d8bb9b  0.000626
107294  C_ID_caf7b48420  0.000626
14139   C_ID_e6526efe1a  0.000626
41833   C_ID_5b4a93e895  0.000626
89033   C_ID_588e756b11  0.000626
7908    C_ID_2d4dfea23e  0.000626
39179   C_ID_2a22ad58ed  0.000626
36418   C_ID_aae3296122  0.000626
24709   C_ID_ff2ba2c76e  0.000626
53101   C_ID_8a060d9a78  0.000626
25743   C_ID_53c0452072  0.000626
111521  C_ID_b49c7c5596  0.000626
49934   C_ID_6af590b066  0.000626
118573  C_ID_fedb7195c4  0.000625
106230  C_ID_300542a511  0.000625
27389   C_ID_0d57cb1cae  0.000625
106400  C_ID_4376c7cbf1  0.000625
77799   C_ID_065cf114be  0.000624
102419  C_ID_20cc14026f  0.000624
70501   C_ID_fcb1a1b1f5  0.000624
13780   C_ID_2feda71c61  0.000624
98145   C_ID_64f5453ef2  0.000623
23009   C_ID_e73bdcca6c  0.000623
108906  C_ID_ef0699c1ed  0.000621

[123623 rows x 2 columns]

In [2]:
import pandas as pd
data =pd.read_csv("result/LGB_ParamSearch_groupby_submission.csv")

data2 =pd.read_csv("result/LGB_ParamSearch_groupby_Drop_submission.csv")

data['target2'] = data2['target'].values
print(data.corr())

data['target'] = data['target']*0.8+data['target2']*0.2
data[['card_id','target']].to_csv("result/groupby8_groupbyDrop2.csv", index=False)

           target   target2
target   1.000000  0.434902
target2  0.434902  1.000000


In [9]:
import pandas as pd
sub1 = pd.read_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6.csv")
sub2 = pd.read_csv("result/LGB_ParamSearch_groupby_Class_submission.csv")
sub = sub1.copy()
for k in range(30, 100, 10):
    card = sub2.sort_values("target", ascending=False)[:k]
    card['target'] = card['target']*(-33.21928095)
    sub['target2'] = sub['card_id'].map(pd.Series(card['target'].values, index=card['card_id'].values)).fillna(0)
    sub['target'] = sub['target']*((sub['target2']==0).astype(int)) + sub['target2']
    print(sub['target'].value_counts().head(10))
    sub[sub1.columns].to_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6_top"+str(k)+".csv", index=False)

 0.403289    2
-0.350197    1
-1.208321    1
-0.182470    1
-0.080934    1
 0.151976    1
 0.203276    1
-0.392778    1
-0.108030    1
 0.809024    1
Name: target, dtype: int64
 0.403289    2
-0.350197    1
 0.533485    1
-0.080934    1
 0.151976    1
 0.203276    1
-0.392778    1
-0.108030    1
 0.809024    1
-1.660821    1
Name: target, dtype: int64
 0.403289    2
-0.350197    1
 0.588402    1
-0.257936    1
-0.182470    1
-0.080934    1
 0.151976    1
 0.203276    1
-0.392778    1
-0.108030    1
Name: target, dtype: int64
 0.403289    2
-0.350197    1
 0.301347    1
 0.151976    1
 0.203276    1
-0.392778    1
-0.108030    1
 0.809024    1
-1.660821    1
-0.948698    1
Name: target, dtype: int64
 0.403289    2
-0.350197    1
-0.655424    1
 0.525933    1
-0.257936    1
-0.182470    1
-0.080934    1
 0.151976    1
 0.203276    1
-0.392778    1
Name: target, dtype: int64
 0.403289    2
-0.350197    1
 0.203276    1
 0.870073    1
-0.777764    1
-1.184682    1
-6.020541    1
-0.111196 

In [2]:
import pandas as pd
data =pd.read_csv("result/LGB_ParamSearch_groupby_submission.csv")

data2 =pd.read_csv("result/XGB_ParamSearch_submission.csv")

data['target2'] = data2['target'].values
print(data.corr())

data['target'] = data['target']*0.8+data['target2']*0.2
data[['card_id','target']].to_csv("result/XGB_ParamSearch8_LGB_ParamSearch_groupby2.csv", index=False)

           target   target2
target   1.000000  0.850789
target2  0.850789  1.000000
